In [1]:
#https://cryptographycaffe.sandboxaq.com/posts/kyber-01/
#https://www.cybersecurity.blog.aisec.fraunhofer.de/en/a-somewhat-gentle-introduction-to-lattice-based-post-quantum-cryptography/
#https://www.youtube.com/watch?v=lVQuV1sfSw4

In [2]:
import numpy as np
from numpy.polynomial import polynomial as poly
import random

In [3]:
def parameters():
    # polynomial modulus degree
    n = 2**2
    # ciphertext modulus
    q = 67
    # polynomial modulus
    poly_mod = np.array([1] + [0] * (n - 1) + [1])
    #module rank
    k = 2
    return (n,q,poly_mod,k)

In [4]:
def polyadd(x, y, modulus, poly_mod):
    """Multiply two polynoms
    Args:
        x, y: two polynoms to be multiplied.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        A polynomial in Z_modulus[X]/(poly_mod).
    """
    return np.int64(
        np.round(poly.polydiv(poly.polyadd(x, y) % modulus, poly_mod)[1] % modulus)
    )

def polymul(x, y, modulus, poly_mod):
    """Add two polynoms
    Args:
        x, y: two polynoms to be added.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        A polynomial in Z_modulus[X]/(poly_mod).
    """
    return np.int64(
        np.round(poly.polydiv(poly.polymul(x, y) % modulus, poly_mod)[1] % modulus)
    )

#additive inverse of polynomial a mod q
def inv_poly(a, modulus):
  return list(map(lambda x: -x % modulus, a))

#subtract polynomials a, b mod q
def sub_poly(a, b, modulus, poly_mod):
  return polyadd(a, inv_poly(b, modulus), modulus, poly_mod)

In [5]:
np.random.seed(0xdeadbeef)

def sign_extend(poly, degree):
  if len(poly) >= degree:
    return poly
  
  return [0] * (degree - len(poly))

def test_mul_poly(N, f, q):
  degree_f = len(f) - 1

  for i in range(N):
    a = (np.random.random(degree_f) * q).astype(int)
    b = (np.random.random(degree_f) * q).astype(int)
    
    a_mul_b = polymul(a, b, q, f)
    a_mul_b.resize(degree_f)
    
    # NumPy reference poly multiplication
    # note that we need to convert the coefficients to int and extend the list to match the fixed size of our impl
    a_mul_b_ref = list(map(lambda x: int(x) % q, ((poly.Polynomial(a) * poly.Polynomial(b)) % poly.Polynomial(f)).coef))
    a_mul_b_ref = sign_extend(a_mul_b_ref, degree_f)

    assert(a_mul_b.tolist() == a_mul_b_ref)

test_mul_poly(100, [1, 0, 0, 0, 1], 17)

In [6]:
def add_vec(v0, v1, q):
  assert(len(v0) == len(v1)) # sizes need to be the same

  result = []

  for i in range(len(v0)):
    result.append(polyadd(v0[i], v1[i], q, f))
  
  return result


def mul_vec_simple(v0, v1, f, q):
  assert(len(v0) == len(v1)) # sizes need to be the same

  degree_f = len(f) - 1
  result = [0 for i in range(degree_f - 1)]

  # textbook vector inner product
  for i in range(len(v0)):
    result = polyadd(result, polymul(v0[i], v1[i], q, f), q, f)
  
  return result


def mul_mat_vec_simple(m, a, f, q):
  result = []
  
  # textbook matrix-vector multiplication
  for i in range(len(m)):
    result.append(mul_vec_simple(m[i], a, f, q))
  
  return result


def transpose(m):
  result = [[None for i in range(len(m))] for j in range(len(m[0]))]

  for i in range(len(m)):
    for j in range(len(m[0])):
      result[j][i] = m[i][j]
  
  return result

In [7]:
np.random.seed(0xdeadbeef)

def test_mul_vec(N, k, f, q):
  degree_f = len(f) - 1

  for i in range(N):
    m = (np.random.random([k, k, degree_f]) * q).astype(int)
    a = (np.random.random([k, degree_f]) * q).astype(int)

    m_mul_a = mul_mat_vec_simple(m, a, f, q)
    m_mul_a = [np.pad(a,(0,degree_f - len(a))) for a in m_mul_a] #pad the results on the right

    m_poly = list(map(lambda x: list(map(poly.Polynomial, x)), m))
    a_poly = list(map(poly.Polynomial, a))
    prod = np.dot(m_poly, a_poly)
    m_mul_a_ref = list(map(lambda x: list(map(lambda y: int(y) % q, sign_extend((x % poly.Polynomial(f)).coef, degree_f))), prod))
    m_mul_a_ref = [np.int64(a) for a in m_mul_a_ref] #convert the reference to np.int64
      
    assert all(np.array_equal(a, b) for a, b in zip(m_mul_a, m_mul_a_ref))

test_mul_vec(100, 2, [1, 0, 0, 0, 1], 17)

In [8]:
def encrypt(A, t, m_b, f, q, r, e_1, e_2):
  half_q = int(q / 2 + 0.5)
  m = list(map(lambda x: x * half_q, m_b))

  u = add_vec(mul_mat_vec_simple(transpose(A), r, f, q), e_1, q)
  v = sub_poly(polyadd(mul_vec_simple(t, r, f, q), e_2, q, f), m, q, f)

  return u, v

In [9]:
def decrypt(s, u, v, f, q):
  m_n = sub_poly(v, mul_vec_simple(s, u, f, q), q, f)

  half_q = int(q / 2 + 0.5)
  def round(val, center, bound):
    dist_center = np.abs(center - val)
    dist_bound = min(val, bound - val)
    return center if dist_center < dist_bound else 0

  m_n = list(map(lambda x: round(x, half_q, q), m_n))
  m_b = list(map(lambda x: x // half_q, m_n))
  
  return m_b

In [10]:
# Baby Kyber params
q = 17 # plain modulus
f = [1, 0, 0, 0, 1] # poly modulus, x**4 + 1

s = [[0, 1, -1, -1], [0, -1, 0, -1]] # secret key, [-x**3-x**2+x, -x**3-x]
A = [[[11, 16, 16, 6], [3, 6, 4, 9]], [[1, 10, 3, 5], [15, 9, 1, 6]]] # public key
e = [[0, 0, 1, 0], [0, -1, 1, 0]] # noise
m_b = [1, 1, 0, 1] # message in binary

t = add_vec(mul_mat_vec_simple(A, s, f, q), e, q)

r = [[0, 0, 1, -1], [-1, 0, 1, 1]] # blinding vector for encrypt
e_1 = [[0, 1, 1, 0], [0, 0, 1, 0]] # noise vector for encrypt
e_2 = [0, 0, -1, -1] # noise poly for encrypt

u, v = encrypt(A, t, m_b, f, q, r, e_1, e_2)
m_b2 = decrypt(s, u, v, f, q)

assert(m_b == m_b2)

In [13]:
np.random.seed(0xdeadbeef)
import json

def test_enc_dec(N, k, f, q):
  degree_f = len(f) - 1

  A = (np.random.random([k, k, degree_f]) * q).astype(int) #note A \in R^{k x k}, each entry is a deg(f) list
  s = (np.random.random([k, degree_f]) * 3).astype(int) - 1 #each coefficient is in {-1,0,+1}
  e = (np.random.random([k, degree_f]) * 3).astype(int) - 1 #each coefficient is in {-1,0,+1}
  t = add_vec(mul_mat_vec_simple(A, s, f, q), e, q)

  failed = 0

  for i in range(N):
    m_b = (np.random.random(degree_f) * 2).astype(int)

    r = (np.random.random([k, degree_f]) * 3).astype(int) - 1
    e_1 = (np.random.random([k, degree_f]) * 3).astype(int) - 1
    e_2 = (np.random.random([degree_f]) * 3).astype(int) - 1

    u, v = encrypt(A, t, m_b, f, q, r, e_1, e_2)
    print(u,v)
    m_b2 = decrypt(s, u, v, f, q)
    m_b2 = np.pad(m_b2,(0,degree_f - len(m_b2))) #zero pad result

    if m_b.tolist() != m_b2.tolist():
      failed += 1
  
  print(f"[k={k}, f={f}, q={q}] Test result: {failed}/{N} failed decryption!")

test_enc_dec(100, 2, [1, 0, 0, 0, 1], 17)
test_enc_dec(100, 2, [1, 0, 0, 0, 1], 37)
test_enc_dec(100, 2, [1, 0, 0, 0, 1], 67)

[array([ 8,  2,  7, 12]), array([ 0,  3, 14,  4])] [10  8 11  6]
[array([14,  7,  7]), array([ 4,  9,  7, 12])] [ 3 12 12  4]
[array([14,  4, 11, 11]), array([15, 11,  8, 16])] [4 7 1 3]
[array([8, 7, 0, 9]), array([10,  3,  7, 10])] [6 5 9 6]
[array([13, 11,  9, 14]), array([ 6,  7,  1, 16])] [12  8 10  7]
[array([ 7,  4, 14, 13]), array([10,  5,  9])] [ 7  2  5 16]
[array([ 4,  8,  0, 11]), array([ 3, 15,  3, 12])] [ 9  6 11  5]
[array([ 6,  0, 15,  7]), array([3, 6, 7, 4])] [ 9  7 13  2]
[array([15, 16,  3, 12]), array([ 3,  8, 11, 14])] [ 1 10 11 10]
[array([10,  0, 16,  4]), array([ 4,  9,  2, 14])] [ 3  6 11 11]
[array([ 5,  8, 12,  6]), array([ 4, 13, 14, 11])] [10 11 12  8]
[array([8, 3, 8, 7]), array([11,  5, 15, 13])] [4 6 5 6]
[array([6, 3, 4, 5]), array([3, 3, 3, 9])] [ 4  5  6 10]
[array([13,  7, 16,  3]), array([ 0,  5,  9, 14])] [14  8  1 14]
[array([ 4, 11,  5, 12]), array([5, 9, 7, 7])] [ 0 10  2 15]
[array([ 9,  4,  2, 12]), array([10, 13, 12,  1])] [ 4  3 11  1]
[arr